In [7]:
"""This sample shows how to shows how to record data to an .rrf file.

This sample uses Royale's feature of stopping after a given number of
frames are captured, therefore the --frames argument is required.
"""

import argparse
import queue
import roypy
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options
#from roypy_platform_utils import PlatformHelper

class MyListener (roypy.IRecordStopListener):
    """A simple listener, in which waitForStop() blocks until onRecordingStopped has been called."""
    def __init__ (self):
        super (MyListener, self).__init__()
        self.queue = queue.Queue()

    def onRecordingStopped (self, frameCount):
        self.queue.put (frameCount)

    def waitForStop (self):
        frameCount = self.queue.get()
        print ("Stopped after capturing {frameCount} frames".format (frameCount=frameCount))

def main ():
    #platformhelper = PlatformHelper()
    parser = argparse.ArgumentParser (usage = __doc__)
    add_camera_opener_options (parser)
    parser.add_argument ("--frames", type=int, required=True, help="duration to capture data (number of frames)")
    parser.add_argument ("--output", type=str, required=True, help="filename to record to")
    parser.add_argument ("--skipFrames", type=int, default=0, help="frameSkip argument for the API method")
    parser.add_argument ("--skipMilliseconds", type=int, default=0, help="msSkip argument for the API method")
    #options = parser.parse_args()
    options = parser.parse_args(args=['--frames', '75', '--output', '456.rrf'])#,'--skipMilliseconds','1000'

    opener = CameraOpener (options)
    cam = opener.open_camera ()

    print_camera_info (cam)

    l = MyListener()
    cam.registerRecordListener(l)
    cam.startCapture()
    cam.startRecording (options.output, options.frames, options.skipFrames, options.skipMilliseconds);

    seconds = options.frames * (options.skipFrames + 1) / cam.getFrameRate()
    if options.skipMilliseconds:
        timeForSkipping = options.frames * options.skipMilliseconds / 1000
        seconds = int (max (seconds, timeForSkipping))

    print ("Capturing with the camera running at {rate} frames per second".format (rate=cam.getFrameRate()))
    print ("This is expected to take around {seconds} seconds".format (seconds=seconds))
    
    l.waitForStop()

    cam.stopCapture()

if (__name__ == "__main__"):
    main()


Number of cameras connected:  1
        Camera information
Type:            PICOFLEXX
Width:           224
Height:          171
Operation modes: 10
    MODE_9_5FPS_2000
    MODE_9_10FPS_1000
    MODE_9_15FPS_700
    MODE_9_25FPS_450
    MODE_5_35FPS_600
    MODE_5_45FPS_500
    MODE_MIXED_30_5
        this operation mode has 2 streams
    MODE_MIXED_50_5
        this operation mode has 2 streams
    Low_Noise_Extended
    Fast_Acquisition
CameraInfo items: 8
    ('BRIDGE_TYPE', 'Enclustra')
    ('MODULE_IDENTIFIER', '00000000')
    ('MODULE_IDENTIFIER_HASH', '558161692')
    ('MODULE_SERIAL', '0')
    ('MODULE_SUFFIX', '')
    ('IMAGER', 'M2450_A12_AIO')
    ('PROCESSING_NAME', 'Spectre')
    ('PROCESSING_VERSION', '3.13.2.773')
Capturing with the camera running at 5 frames per second
This is expected to take around 15.0 seconds
Stopped after capturing 75 frames
